In [13]:
# importing the packages

import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

In [43]:
#importing the dataset

data = pd.read_csv(r'C:\Users\Grade A\Desktop\DATASET\data science africa\Python-Data-Basics\dataset_original.csv')

In [44]:
#checking the first five rows of the dataset

data.head()

,feature_1,feature_2,label
0,1.147868,0.937782,1
1,0.173189,-0.264483,0
2,0.680447,0.048147,1
3,-1.088036,1.028807,1
4,1.193379,-1.155007,0


In [45]:
# splitting & dropping the target varaiable into X & y

X = data.drop('label', axis=1)
y = data.label

In [46]:
# calling in the (AUC) function

from sklearn.metrics import auc

In [47]:
#splitting the dataset into train and test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [48]:
# calling in the (AUC) function

from sklearn.metrics import auc

In [49]:
#tuning my model

model=CatBoostClassifier(n_estimators=800,eval_metric='AUC',max_depth=5,learning_rate=0.1,od_wait=50, 
                              subsample=0.9,bootstrap_type='Bernoulli',metric_period=20,
                     #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     use_best_model=True)

In [50]:
#fitting my model

model.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	test: 0.9374213	best: 0.9374213 (0)	total: 5.11ms	remaining: 4.08s
20:	test: 0.9736558	best: 0.9751843 (17)	total: 84.9ms	remaining: 3.15s
40:	test: 0.9717677	best: 0.9751843 (17)	total: 157ms	remaining: 2.9s
60:	test: 0.9696098	best: 0.9751843 (17)	total: 238ms	remaining: 2.88s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9751843194
bestIteration = 17

Shrink model to first 18 iterations.


In [51]:
#saving my prediction file

pred = model.predict_proba(data)[:, 1]

In [52]:
#splitting my model for further tuning

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [53]:
# my first tuning model

from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb=[]
y_pred_totcb=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit, GroupKFold
fold=StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m=CatBoostClassifier(n_estimators=800,eval_metric='AUC',max_depth=4,learning_rate=0.1,#reg_lambda=5,#5
                              subsample=0.9,bootstrap_type='Bernoulli',#leaf_estimation_iterations=10,
                    #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     use_best_model=True)
    #CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=5,learning_rate=0.1,reg_lambda=5,#5
                              #subsample=0.9,bootstrap_type='Bernoulli',
                    #l2_leaf_reg=5,#bagging_temperature=0.85,random_strength=100,
                     #use_best_model=True)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(data)[:, 1]
    y_pred_totcb.append(p)

0:	test: 0.9655373	test1: 0.9627093	best: 0.9627093 (0)	total: 6.36ms	remaining: 5.08s
100:	test: 0.9981693	test1: 0.9852092	best: 0.9859981 (19)	total: 358ms	remaining: 2.47s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9859980638
bestIteration = 19

Shrink model to first 20 iterations.
err:  0.985998063752734
0:	test: 0.9696013	test1: 0.9434745	best: 0.9434745 (0)	total: 4.21ms	remaining: 3.37s
100:	test: 0.9975275	test1: 0.9908376	best: 0.9908376 (99)	total: 388ms	remaining: 2.69s
200:	test: 0.9998292	test1: 0.9907294	best: 0.9909097 (110)	total: 789ms	remaining: 2.35s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9909097468
bestIteration = 110

Shrink model to first 111 iterations.
err:  0.9909097467715171
0:	test: 0.9588391	test1: 0.9783746	best: 0.9783746 (0)	total: 4.43ms	remaining: 3.54s
100:	test: 0.9980714	test1: 0.9879879	best: 0.9899358 (29)	total: 380ms	remaining: 2.63s
Stopped by overfitting detector  (100 iterations wait)

b

In [54]:
#checking the np.mean(errb)


np.mean(errcb)

0.9889478671975721

In [55]:
#checking the second tuning model

from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
errcb1=[]
y_pred_totcb1=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
fold=StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    m1=CatBoostClassifier(n_estimators=1000,eval_metric='AUC',max_depth=4,learning_rate=0.1,od_wait=50, reg_lambda=3,
                              bootstrap_type='Bayesian',
                     use_best_model=True)
    m1.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m1.predict_proba(X_test)[:, 1]
    print("err: ",roc_auc_score(y_test,preds))
    errcb1.append(roc_auc_score(y_test,preds))
    p1 = m1.predict_proba(data)[:, 1]
    y_pred_totcb1.append(p1)


0:	test: 0.9726751	test1: 0.9657930	best: 0.9657930 (0)	total: 4.41ms	remaining: 4.41s
100:	test: 0.9979799	test1: 0.9842052	best: 0.9873606 (3)	total: 525ms	remaining: 4.67s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9873606081
bestIteration = 3

Shrink model to first 4 iterations.
err:  0.9873606081250672
0:	test: 0.9771943	test1: 0.9672282	best: 0.9672282 (0)	total: 4.56ms	remaining: 4.55s
100:	test: 0.9969251	test1: 0.9887995	best: 0.9889077 (93)	total: 518ms	remaining: 4.61s
200:	test: 0.9994785	test1: 0.9905310	best: 0.9908556 (198)	total: 975ms	remaining: 3.88s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9908556381
bestIteration = 198

Shrink model to first 199 iterations.
err:  0.9908556381213477
0:	test: 0.9739080	test1: 0.9758856	best: 0.9758856 (0)	total: 4.5ms	remaining: 4.5s
100:	test: 0.9974960	test1: 0.9874468	best: 0.9902063 (33)	total: 522ms	remaining: 4.65s
Stopped by overfitting detector  (100 iterations wait)

bestT

In [58]:
#checking the np.mean(errcb1)

np.mean(errcb1)

0.9894741935219095

In [60]:
# savign my tuned model into test_predictions

d = {"label": np.mean(y_pred_totcb, 0)}
test_predictions = pd.DataFrame(data=d)
test_predictions = test_predictions[['label']]

In [29]:
test_predictions.to_csv('mode.csv', index=False)

In [30]:
#checking the first five rows of my test_predictions

test_predictions.head()

,label
0,0.992398
1,0.122025
2,0.732201
3,0.988941
4,0.011037


In [61]:
#saving my second tuned model

d = {"label": np.mean(y_pred_totcb1, 0)}
test_predictioned = pd.DataFrame(data=d)
test_predictioned = test_predictioned[['label']]

In [62]:
test_predictioned.to_csv('mode2.csv', index=False)

In [63]:
#checking the first five rows of my test_predictioned

test_predictioned.head()

,label
0,0.927316
1,0.191907
2,0.772954
3,0.916195
4,0.080720


In [64]:
#saving my predictions into a & b for test_predictions and test_predictioned respectively

a = pd.read_csv('mode.csv')
b = pd.read_csv('mode2.csv')

In [65]:
#coreelating my predictions

a['label'].corr(b['label'])

0.9987253252202769

In [66]:
#correlating my predictions

test_predictions['label'].corr(test_predictioned['label'])

0.9987253252202769

In [67]:
#blendin my models intp test_predictions

test_predictions['label'] = (a['label'] * 0.69 + b['label'] * 0.35) * round(2)

In [69]:
#stacking my model

test_predictions.to_csv('data science africa_stack1.csv', index=False)

In [70]:
#saving my prediction into data.to_csv

data.to_csv('data science africa.csv', index=False)

In [72]:
#saving my prediction file
pred = model.predict_proba(data)[:, 1]